### 清空无效小文件代码

In [5]:
import os
path='./video/'
file_list=os.listdir(path)
print(f'Total {len(file_list)}')
for filepath in file_list:
    if os.path.getsize(path+filepath)<1000:
            os.remove(path+filepath)
            print(f'Remove {path+filepath}')

Total 10627
Remove ./video/6804336928289262856.mp4
Remove ./video/6717655219502648587.mp4
Remove ./video/7056755222839872802.mp4
Remove ./video/7090125423065025792.mp4
Remove ./video/7127086230491204894.mp4
Remove ./video/7159131149242043652.mp4
Remove ./video/7174400019716508936.mp4
Remove ./video/7101633929693777164.mp4


### 核对文档记录数和视频数

In [1]:
import os
import pandas as pd
data1=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/theme_record.xlsx',dtype=str)
data1=data1.drop_duplicates(subset='aweme_id')
data2=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/total_record.xlsx',dtype=str)
data2=data2.drop_duplicates(subset='aweme_id')
data=pd.DataFrame(pd.concat([data1.aweme_id,data2.aweme_id])).drop_duplicates(subset='aweme_id')
path='./video/'
file_list=os.listdir(path)
print('文档总数',len(set((data.aweme_id.astype(str)+'.mp4').values)),'视频总数',len(set(file_list)))

文档总数 10573 文件总数 10628


### 整理 删除记录里有的但是却没视频文件的(覆盖操作)

In [3]:
delete_record_list=list(set((data.aweme_id.astype(str)+'.mp4').values)-set(file_list))
delete_aweme_list=data[data['aweme_id'].isin([x.replace('.mp4','') for x in delete_record_list])]['aweme_id'].values
theme_info=data1[~data1['aweme_id'].isin(delete_aweme_list)]
total_info=data2[~data2['aweme_id'].isin(delete_aweme_list)]
theme_info.to_excel('./backup/clean_theme_record.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)
total_info.to_excel('./backup/clean_total_record.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)

### 评论数统计

In [1]:
import pandas as pd
data=pd.read_excel('./backup/comment_record.xlsx',dtype=str)
print('主题数',len(data.drop_duplicates(subset='aweme_id',keep='first')),'总评论数',len(data.drop_duplicates(subset='cid',keep='first')),'每个主题平均',82972/4611)

#### 主题爬虫合并操作(合并帖子)

In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
data1=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/clean_theme_record.xlsx',dtype=str)
data1=data1.drop_duplicates(subset='aweme_id')
data2=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/clean_total_record.xlsx',dtype=str)
data2=data2.drop_duplicates(subset='aweme_id')
data1=data1[~data1.aweme_id.isin(data2.aweme_id)]
videoinfo=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/videoinfo_record.xlsx',dtype=str)
userinfo=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/author_record.xlsx',dtype=str)
result_list1=data2[['aweme_id', 'collect_count', 'create_time', 'desc',
       'digg_count', 'duration', 'follower_count', 'gender',
       'location', 'nickname', 'region',
       'share_count','signature', 'uid','video_tag', 'keyword','comment_count']]
result_list1.columns=['aweme_id', 'collect_count', 'create_time', 'desc',
       'digg_count', 'duration', 'follower_count', 'gender',
       'location', 'nickname', 'region','share_count','signature', 'uid','video_tag', 'keyword', 'comment_count']

In [2]:
columns=['aweme_id','author_user_id','desc','duration','keyword','comment_number','nickname','uid','gender','following_count','follower_count','signature','location','collect_count', 'comment_count', 'digg_count', 'play_count','share_count','create_time','region','video_tag']
result_list2=[]
for idx,row in data1.iterrows():
    try:
       temp=row[['aweme_id','author_user_id','desc','duration','keyword','comment_number']].values
       temp2=userinfo[userinfo['uid']==temp[1]][['nickname','uid','gender','following_count','follower_count','signature','ip_location']].head(1).values[0]
       temp3=pd.DataFrame([eval(videoinfo[videoinfo['aweme_id']==row['aweme_id']]['statistics'].values[0])]).iloc[:,2:].values[0]
       temp4=videoinfo[videoinfo['aweme_id']==row['aweme_id']][['create_time','region','video_tag']].values[0]
       temp=np.append(temp,temp2)
       temp=np.append(temp,temp3)
       temp=np.append(temp,temp4)
       result_list2.append(dict(zip(columns,temp)))
    except:
       pass
result_list2=pd.DataFrame(result_list2)
result_list2=result_list2[['aweme_id', 'collect_count', 'create_time', 'desc',
       'digg_count', 'duration', 'follower_count', 'gender',
       'location', 'nickname', 'region',
       'share_count','signature', 'uid','video_tag', 'keyword', 'comment_count']]
result_list2.columns=['aweme_id', 'collect_count', 'create_time', 'desc',
       'digg_count', 'duration', 'follower_count', 'gender',
       'location', 'nickname', 'region','share_count','signature', 'uid','video_tag', 'keyword', 'comment_count']
result_list=pd.concat([result_list1,result_list2])
result_list.drop_duplicates(subset='aweme_id')
result_list.index=range(len(result_list))
# result_list['comment_get']=0

In [3]:
comment_data=pd.read_excel('./backup/comment_record.xlsx',dtype=str)
comment_data=comment_data.drop_duplicates(subset='cid',keep='first')
comment=pd.DataFrame([comment_data.value_counts('aweme_id').index,comment_data.value_counts('aweme_id').values]).T
comment.columns=['aweme_id','comment_get']
result_list=pd.merge(result_list,comment,on='aweme_id',how='outer').iloc[:10266]
result_list.comment_get=result_list.comment_get.fillna(0)
# result_list.to_excel('./backup/merge_record.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)

In [5]:
result_list.to_excel('./backup/merge_record.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)

#### 筛选倾向词语

In [1]:
### 可以利用神经网络文本意图（intent）识别,但是数据集只找到了英文的
import pandas as pd
data=pd.read_excel('./backup/comment_recoad_modify.xlsx',dtype=str)
comment_pd=data[['aweme_id','text']]
comment_pd=comment_pd.dropna()

In [2]:
intent_word='|'.join(['要','想','我也','一起','想问','开始','一定','跟着','运动','坚持','愿意','计划','行动','努力'])
# intent_word='|'.join(['愿意','计划','行动','努力'])
select_intent_pd=comment_pd[comment_pd.text.str.contains(intent_word)]
select_intent_pd

,aweme_id,text
4,6600163161221893379,认识你的第一个视频，这时候的你很棒，现在也要加油[奋斗]
6,6600163161221893379,你激励了我，我会像你学习好好努力！！[碰拳]
8,6600163161221893379,周哥从这里开始认识你
10,6600163161221893379,这真的是每一个爱篮球的男生想要的生活了
22,6600163161221893379,是我想要的生活了 难受
...,...,...
122884,7188361515379035404,虽然我也是热身 但我真不信评论区里有这么多[看]
122891,7188361515379035404,只能25[流泪][流泪][流泪] 30还需努力
122896,7188361515379035404,我要追求重量就去玩力量举，你体型倒是没练出来，力量确实相对你这个体型已经不错了
122899,7188361515379035404,30公斤，我感觉我受到了侮辱，要不是我听错了我还以为是30公斤侧平举呢。这里是抖音诶，30公...


In [3]:
merge_data=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/arousal_record.xlsx',dtype=str)
intention_pd=pd.DataFrame([select_intent_pd['aweme_id'].value_counts().index,select_intent_pd['aweme_id'].value_counts().values]).T
merge_data=merge_data.drop('intention_counts',axis=1)
intention_pd.columns=['aweme_id','intention_counts']
result_list=pd.merge(merge_data,intention_pd,on='aweme_id')
result_list.columns
# coment_counter=pd.read_excel('./backup/merge_record.xlsx', dtype=str)
# result_list=pd.merge(result_list,coment_counter[['aweme_id','comment_count']],on='aweme_id')
result_list=result_list.drop('comment_get',axis=1)
comment_info=pd.DataFrame([comment_pd.value_counts('aweme_id').index,comment_pd.value_counts('aweme_id').values]).T
comment_info.columns=['aweme_id','comment_get']
result_list=pd.merge(result_list,comment_info,on='aweme_id',how='inner')
result_list.comment_get=result_list.comment_get.fillna(0)
result_list=result_list[['aweme_id', 'collect_count', 'create_time', 'desc', 'digg_count',
       'duration', 'follower_count', 'gender', 'location', 'nickname',
       'region', 'share_count', 'signature', 'uid', 'video_tag', 'keyword',
       'comment_count', 'comment_get', 'variation', 'arousal',
       'intention_counts']]
result_list.to_excel('./backup/arousal_record_modify.xlsx', engine="xlsxwriter", encoding="utf-8", index=False)

In [6]:
result_list[result_list['aweme_id']=='7180952968378780988'].comment_get

68    4
Name: comment_get, dtype: int64

In [9]:
result_list[result_list['aweme_id']=='7180952968378780988'].intention_counts

68    1
Name: intention_counts, dtype: object

In [10]:
result_list[result_list['aweme_id']=='7180952968378780988'].intention_counts

68    1
Name: intention_counts, dtype: object

In [ ]:
# TODO 博主ID 有用户信息 选粉丝数多的 选减肥vlog 因为他是热播